# DensNet
---

In [1]:
import tensorflow as tf
import numpy as np

## Hyperparameters

In [2]:
EPOCHS = 10

## DenseUnit

In [3]:
class DenseUnit(tf.keras.Model):
    def __init__(self, filter_out, kernel_size):
        super(DenseUnit, self).__init__()
        self.bn = tf.keras.layers.BatchNormalization()
        self.conv = tf.keras.layers.Conv2D(filter_out, kernel_size,
                                          padding = 'same')
        self.concat = tf.keras.layers.Concatenate()
        
    def call(self, x, training = False, mask = None): # (Batch, H, W, Ch_in)
        h = self.bn(x, training = training)
        h = tf.nn.relu(h)
        h = self.conv(h) # h: (Batch, H, W, filter_output)
        return self.concat([x, h]) # (Batch, H, W, Ch_in + filter_output)

## DenseLayer

In [4]:
class DenseLayer(tf.keras.Model):
    def __init__(self, num_unit, growth_rate, kernel_size):
        super(DenseLayer, self).__init__()
        self.sequence = list()
        for idx in range(num_unit):
            self.sequence.append(DenseUnit(growth_rate, kernel_size))
        
    def call(self, x, training = False, mask = None):
        for unit in self.sequence:
            x = unit(x, training = training)
        return x

## Transition Layer

In [5]:
class TransitionLayer(tf.keras.Model):
    def __init__(self, filters, kernel_size):
        super(TransitionLayer, self).__init__()
        self.conv = tf.keras.layers.Conv2D(filters, kernel_size,
                                          padding = 'same')
        self.pool = tf.keras.layers.MaxPool2D()
        
    def call(self, x, training = False, mask = None):
        x = self.conv(x)
        return self.pool(x)

## Model

In [6]:
class DenseNet(tf.keras.Model):
    def __init__(self):
        super(DenseNet, self).__init__()
        # 28 X 28 X 8
        self.conv1 = tf.keras.layers.Conv2D(8, (3, 3), padding = 'same',
                                           activation = 'relu')
        # 28 X 28 X 16
        self.dl1 = DenseLayer(2, 4, (3, 3))
        # 14 X 14 X 16
        self.tr1 = TransitionLayer(16, (3, 3))
        
        # 14 X 14 X 32
        self.dl2 = DenseLayer(2, 8, (3, 3))
        # 7 X 7 X 32
        self.tr2 = TransitionLayer(32, (3, 3))
        
        # 7 X 7 X 64
        self.dl3 = DenseLayer(2, 16, (3, 3))
        
        self.flattened = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(128, activation = 'relu')
        self.dense2 = tf.keras.layers.Dense(10, activation = 'softmax')
        
    def call(self, x, training = False, mask = None):
        x = self.conv1(x)
        
        x = self.dl1(x, training = training)
        x = self.tr1(x)
        
        x = self.dl2(x, training = training)
        x = self.tr2(x)
        
        x = self.dl3(x, training = training)
        
        x = self.flattened(x)
        x = self.dense1(x)
        return self.dense2(x)

## Training and Test Loops

In [7]:
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(images, training = True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)
    
@tf.function
def test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy):
    predictions = model(test_images, training = False)
    t_loss = loss_object(test_labels, predictions)
    
    test_loss(t_loss)
    test_accuracy(test_labels, predictions)

## Dataset

In [8]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train[..., tf.newaxis].astype(np.float32)
x_test = x_test[..., tf.newaxis].astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

## Training Environment

In [9]:
# Creating a model
model = DenseNet()

# Loss Function
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

# Optimization Algorithm
optimizer = tf.keras.optimizers.Adam()

# Evaluation Metrics
train_loss = tf.keras.metrics.Mean(name = 'train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'train_accuracy')
test_loss = tf.keras.metrics.Mean(name = 'test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'test_accuracy')

## Training

In [10]:
for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer,
                   train_loss, train_accuracy)
        
    for test_images, test_labels in test_ds:
        test_step(model, test_images, test_labels, loss_object,
                  test_loss, test_accuracy)
        
    print(f'Epoch {epoch + 1}, ' +
         f'Loss: {train_loss.result()}, ' +
         f'Accuracy: {train_accuracy.result() * 100}, ' +
         f'Test Loss: {test_loss.result()}, ' +
         f'Test Accuracy: {test_accuracy.result() * 100}')
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.11624883115291595, Accuracy: 96.45166015625, Test Loss: 0.08343023806810379, Test Accuracy: 97.48999786376953
Epoch 2, Loss: 0.05719474330544472, Accuracy: 98.34832763671875, Test Loss: 0.0844055786728859, Test Accuracy: 97.75999450683594
Epoch 3, Loss: 0.04723915085196495, Accuracy: 98.65166473388672, Test Loss: 0.08393025398254395, Test Accuracy: 97.97999572753906
Epoch 4, Loss: 0.03860418125987053, Accuracy: 98.86666870117188, Test Loss: 0.04223977401852608, Test Accuracy: 98.8699951171875
Epoch 5, Loss: 0.037367191165685654, Accuracy: 98.961669921875, Test Loss: 0.0545039027929306, Test Accuracy: 98.66999816894531
Epoch 6, Loss: 0.02900741994380951, Accuracy: 99.15833282470703, Test Loss: 0.07778466492891312, Test Accuracy: 98.50999450683594
Epoch 7, Loss: 0.027331821620464325, Accuracy: 99.2550048828125, Test Loss: 0.06474492698907852, Test Accuracy: 98.76000213623047
Epoch 8, Loss: 0.024351460859179497, Accuracy: 99.34500122070312, Test Loss: 0.06305981427431107,